<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/main/tutorials/W1D3_GeneralizedLinearModels/instructor/W1D3_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/tutorials/W1D3_GeneralizedLinearModels/instructor/W1D3_Intro.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# Intro

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

## Overview

Hi folks, welcome to the ‘GLM’ day, the highlight of neuromatch (really)!

We will talk about generalized linear models, the so-called GLMs. The great thing about GLMs is that they provide a simple and elegant framework that unifies several useful ML algorithms and allow you to tackle a wide range of data analysis questions in neuroscience. Think of them as the Swiss Army Knife of Machine Learning algorithms.

In particular, some algorithms you might know, such as linear regression, logistic regression, and Poisson regression, are all just special cases of GLMs.  They are very useful if you want to study encoding (i.e., predict neural activity from external covariates) or decoding (i.e., predict a behavior or intention from neural activity).

In addition, GLMs are great for teaching some general concepts in machine learning that are useful to know-- optimization of cost functions (and what is means for a cost function to be convex, and why that is useful), what overfitting is, how to spot it (cross-validation!) and how to protect against it (regularization!), what types of regularization exist (L1, L2), and how this connects to Reverend Bayes.

Finally -- and just like a Swiss Army Knife’, GLMs have limitations, and sometimes we will need more powerful methods-- however, even in those cases, GLMs provide a baseline against which more complicated algorithms (e.g., deep nets) can be compared. In addition, GLMs can be used as building blocks for constructing more complicated algorithms.

The day is structured as follows:
In the ‘Intro’, Dr. Christina Savin will first give an overview of canonical analysis tasks in neuroscience and show what they have in common and how they can be tackled with GLMs. She will explain one task (receptive field fitting) as a running example and use it to explain concepts like optimization, overfitting, and regularization.

In the first part of Tutorial 1, Dr. Anqi Qu (now an assistant prof at Georgia Tech!) will delve deeper into our running example (receptive field estimation), and go through the details of the mathematics of this task, and show that spike-triggered averaging (a great but often heuristically motivated approach) is closely related to a special case of GLMs-- the good old linear regression. In the second part of Tutorial 2, she will show how ‘Poisson GLMs’ is a really powerful way to fit encoding models to spiking data.

In Tutorial 3, she will turn to logistic regression (my favorite classification algorithm) and show how it can be used for decoding. She will also explain regularization and the difference between different regularizers.
In the ‘Outro’, Dr. Memming Park will first tell you about his cat, and go into much more detail  about applications of GLMs in neuroscience- after a general overview, he will talk about how GLMs can be used to model the dynamics of single neurons and neural population dynamics, and how they can be used to model the responses of neurons in decision-making tasks and to decode behavioural choices from neurons. Finally-- but importantly-- he will talk about some potential pitfalls in interpreting GLMs, particularly when the danger of (falsely) interpreting their parameters in a causal manner. In the end, he will touch on some possible generalizations of GLMs.

Regarding the overall course, the closest connections are with the day of ‘model fitting’, which will already introduce some of the basic ideas. Once you have been through the GLM day, you will then likely also notice the use of GLMs in many other days and tasks (even if they might not always be referred to as ‘GLMs’), e.g., for decoding behavioral data, or as building blocks as models of dynamics.

So, enjoy the day!

## Prerequisite knowledge

In today's tutorials, you will use matrix vector multiplication ([W0D3 T2](https://compneuro.neuromatch.io/tutorials/W0D3_LinearAlgebra/student/W0D3_Tutorial2.html)), probability distributions ([W0D5 T1](https://compneuro.neuromatch.io/tutorials/W0D5_Statistics/student/W0D5_Tutorial1.html)), and maximum likelihood estimates ([W0D5 T2](https://compneuro.neuromatch.io/tutorials/W0D3_LinearAlgebra/student/W0D3_Tutorial2.html) and [W1D3](https://compneuro.neuromatch.io/tutorials/W1D3_ModelFitting/chapter_title.html)).  You will also be looking at stimulus response pairings ([Neuro Video Series Stimulus Representation](https://compneuro.neuromatch.io/tutorials/W0D0_NeuroVideoSeries/student/W0D0_Tutorial10.html)) and decision making experiments in animals ([Neuro Video Series Behavioural Readouts](https://compneuro.neuromatch.io/tutorials/W0D0_NeuroVideoSeries/student/W0D0_Tutorial3.html)). You will use a new Python library called [sklearn](https://scikit-learn.org/stable/).

In [ ]:
# @title Install and import feedback gadget

!pip3 install vibecheck datatops --quiet

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_cn",
            "user_key": "y1x3mpx5",
        },
    ).render()


feedback_prefix = "W1D3_Intro"

## Video

In [ ]:
# @markdown
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'm1w7oywzwpA'), ('Bilibili', 'BV1BK411H7ie')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Slides

In [ ]:
# @markdown
from IPython.display import IFrame
link_id = "qxfz9"
print(f"If you want to download the slides: https://osf.io/download/{link_id}/")
IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/{link_id}/?direct%26mode=render%26action=download%26mode=render", width=854, height=480)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_Day_Intro")